**Importing Libraries and accessing Files**

In [17]:
import nltk
nltk.download('wordnet')
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
import re

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
data=pd.read_csv("/content/drive/MyDrive/nairaland/nairaland_data.csv",names=["title","content","views","sections"])
common_used_words=open("/content/drive/MyDrive/nairaland/1000mostusedenglishwords.txt","r")

In [13]:
data

,title,content,views,sections
0,"Did Wike Vacate His Seat, When He Was Sacked B...",Governor Wike was also sacked by the Tribunal...,(353 Views),Politics
1,RCCG: The Delusion Of Broda Dele On Osinbajo P...,By Kehinde AdeoyeWhen last did Broda Dele Momo...,(450 Views),Politics
2,"Don't Bill Me, Petrol Is Expensive In Germany ...","Actress, Olaide Oyedeji, has urged her fans to...",(932 Views),Politics
3,The Russia-ukraine War Exposes Why African Nat...,"I keep asking my self,if a soveriegn European ...",(268 Views),Politics
4,APC Caretaker Chairman Mai Mala Buni Begs Buhari,Buni to travel to London to plead with Buhari....,(561 Views),Politics
...,...,...,...,...
92954,Learn Adobe Photoshop And Adobe Illustrator At...,Learn Adobe photoshop and adobe illustrator wi...,(719 Views),"Art, Graphics & Video"
92955,Our Story So Far: Designing For The Law School...,Hello and good morning all! Hope you had an ev...,(1281 Views),"Art, Graphics & Video"
92956,Part Time Graphic Designers Wanted,Work from homeWeekly PayWhatsApp +233543436069...,(828 Views),"Art, Graphics & Video"
92957,Are You A 3D Animator Or Motion Graphic Artist...,Hello Graphic designers. i need of a 3D animat...,(1569 Views),"Art, Graphics & Video"


In [14]:
print(f"columns : {list(data.columns)})")

columns : ['title', 'content', 'views', 'sections'])


In [15]:
data.describe(include="all")

,title,content,views,sections
count,92950,92748,92959,92959
unique,87792,88702,12542,19
top,.,.,(100 Views),Religion
freq,354,545,418,6519


In [16]:
data

,title,content,views,sections
0,"Did Wike Vacate His Seat, When He Was Sacked B...",Governor Wike was also sacked by the Tribunal...,(353 Views),Politics
1,RCCG: The Delusion Of Broda Dele On Osinbajo P...,By Kehinde AdeoyeWhen last did Broda Dele Momo...,(450 Views),Politics
2,"Don't Bill Me, Petrol Is Expensive In Germany ...","Actress, Olaide Oyedeji, has urged her fans to...",(932 Views),Politics
3,The Russia-ukraine War Exposes Why African Nat...,"I keep asking my self,if a soveriegn European ...",(268 Views),Politics
4,APC Caretaker Chairman Mai Mala Buni Begs Buhari,Buni to travel to London to plead with Buhari....,(561 Views),Politics
...,...,...,...,...
92954,Learn Adobe Photoshop And Adobe Illustrator At...,Learn Adobe photoshop and adobe illustrator wi...,(719 Views),"Art, Graphics & Video"
92955,Our Story So Far: Designing For The Law School...,Hello and good morning all! Hope you had an ev...,(1281 Views),"Art, Graphics & Video"
92956,Part Time Graphic Designers Wanted,Work from homeWeekly PayWhatsApp +233543436069...,(828 Views),"Art, Graphics & Video"
92957,Are You A 3D Animator Or Motion Graphic Artist...,Hello Graphic designers. i need of a 3D animat...,(1569 Views),"Art, Graphics & Video"


**Data Cleaning**

In [18]:
#Converting the views column to string
symbols=[" ",")","Views","("]
for symbol in symbols:
  data["views"]=data["views"].str.replace(symbol," ")
data["views"]=data["views"].astype("f")
data=data.sort_values(by="views")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.


In [19]:
#DROPPING DUPLICATES AND DROP EMPTY ROWS
data=data.drop_duplicates()
data=data.dropna()

In [20]:
#GETTING COMMON ENGLISH WORDS TO BE USED AS STOPWORDS
common_words=[]
for word in common_used_words:
  common_words.append(word.replace("\n",""))

In [22]:
def lemmatize_column(column):
  """This lemmatizes every word,removes symbols from the word and removes common english words"""
  lemmatizer=WordNetLemmatizer()
  new_column=[]
  for statement in data[column]:
    statement=re.sub('http[s]?://\S+', '', statement)
    new_statement=[]
    for word in statement.split(" "):
      word=word.replace("(","")
      word=word.replace(")","")
      word=word.replace(".","")
      word=word.replace("&","")
      word=word.replace(",","")
      word=word.replace("'","")
      word=word.replace("/","")
      word=word.replace("\\","")
      word=word.replace("?","")
      if word in common_words:
        pass
      else:
        lemmatized_word=lemmatizer.lemmatize(word)
        new_statement.append(lemmatized_word.lower())
    new_column.append(" ".join(new_statement))
  return new_column

In [23]:
data["title"]=lemmatize_column("title")
data["content"]=lemmatize_column("content")

In [24]:
data=data["title"]+" "+ data["content"]

In [25]:
data.to_pickle("posts.pkl") 